In [18]:
import cv2,cv2 as cv
import numpy as np
import threading
def timer(handler, interval):
    def timer_interrupt():
        # 创建一个 Timer 对象，每隔 interval 秒钟调用一次 timer_interrupt 函数
        t = threading.Timer(interval, timer_interrupt)
        handler()
        # 启动定时器
        t.start()

    # 创建一个 Timer 对象，每隔 1 秒钟调用一次 timer_interrupt 函数
    t = threading.Timer(0.1, timer_interrupt)
    # 启动定时器
    t.start()



In [5]:
import mediapipe as mp
import cv2,cv2 as cv
import numpy as np
import random

def Gesture_recognition(img):
    re_list=[]
    # 将BGR转换为RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 识别图像中的手势，并返回结果
    results = my_hands.process(img)
    # 再将RGB转回BGR
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # results.multi_hand_landmarks为None时进行for循环会报错，所以要先判断
    if results.multi_hand_landmarks:
        for hand_landmark in results.multi_hand_landmarks:
            # hand_drawing_utils.draw_landmarks(img,
            #                                 hand_landmark,
            #                                 mp_hands.HAND_CONNECTIONS,
            #                                 mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
            #                                 mp.solutions.drawing_styles.get_default_hand_connections_style())
            distans=hand_distans(hand_landmark)
            line1_points=Extract_single_line_points(distans,1)
            line4_points=Extract_single_line_points(distans,4)
            hand_landmark=hand_landmark.landmark
            if max(line4_points)>1.5*np.mean(line4_points):
                print('单')
                re_list.append(0)
                re_list.append([hand_landmark[4+4*line4_points.index(max(line4_points))]])
                
            elif np.mean(line4_points) < 1.2*np.mean(line1_points):
                print("握拳")
                re_list.append(1)
                re_list.append([hand_landmark[0]])
            else:
                print('张')
                re_list.append(2)
                re_list.append([hand_landmark[4],hand_landmark[8],hand_landmark[12],hand_landmark[16],hand_landmark[20]])
            return re_list
            
def length_point2(landmark1,landmark2):
    x=landmark1.x-landmark2.x
    y=landmark1.y-landmark2.y
    return (x**2+y**2)**0.5
    
def hand_distans(hand_landmark):
    distans=[0,]
    for i in range(1,21):
        distans.append(length_point2(hand_landmark.landmark[i],hand_landmark.landmark[0]))
    return distans

def Extract_single_line_points(distans,i):
    line_points=[]
    for i in range(i,i+4*4+1,4):
        line_points.append(distans[i])
    return line_points

# 配置meidapipe
hand_drawing_utils = mp.solutions.drawing_utils  # 绘图工具
mp_hands = mp.solutions.hands  # 手部识别api
my_hands = mp_hands.Hands()  # 获取手部识别类


import numpy as np
import math
import cv2,cv2 as cv

def coordinate_transform(point, origin, rotation_angle):
    # 将角度转换为弧度
    rotation_angle_rad = math.radians(rotation_angle)
    
    # 进行坐标变换
    x = point[0] * math.cos(rotation_angle_rad) - point[1] * math.sin(rotation_angle_rad) + origin[0]
    y = point[0] * math.sin(rotation_angle_rad) + point[1] * math.cos(rotation_angle_rad) + origin[1]
    
    return [x,y]
def inverse_coordinate_transform(point, origin, rotation_angle):
    # 将角度转换为弧度
    rotation_angle_rad = math.radians(rotation_angle)
    
    # 进行逆坐标变换
    x_b = (point[0] - origin[0]) * math.cos(rotation_angle_rad) + (point[1] - origin[1]) * math.sin(rotation_angle_rad)
    y_b = -(point[0] - origin[0]) * math.sin(rotation_angle_rad) + (point[1] - origin[1]) * math.cos(rotation_angle_rad)
    
    return [x_b, y_b]
def angle_with_origin(x, y):
    # 使用反三角函数计算夹角（以弧度为单位）
    angle_rad = math.atan2(y, x)
    
    # 将弧度转换为角度
    angle_deg = math.degrees(angle_rad)
    
    adjusted_angle = (angle_deg + 180) % 360 - 180
    return angle_deg
def limit(v,L,H):
    if v<L:
        return L
    elif v>H:
        return H
    return v
def distans_point2(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5
class cars:
    def __init__(self,origin,rotation_angle):
        self.point1=[20,0]
        self.point2=[0,5]
        self.point3=[0,-5]
        self.origin=origin
        self.rotation_angle = rotation_angle
        self.speed=0
        self.dy=0
        self.dx=0
        self.err_angle=0
    def run(self,obj_point):
        L=distans_point2(obj_point,self.origin)
        self.dx-=(obj_point[0]-self.origin[0])*21/L
        self.dy-=(obj_point[1]-self.origin[1])*21/L

        self.dx+=1.0*(obj_point[0]-self.origin[0])
        self.dy+=1.0*(obj_point[1]-self.origin[1])
        angle_rad = math.atan2(self.dy,self.dx)
        angle_deg = math.degrees(angle_rad)
      
        self.err_angle=angle_deg-self.rotation_angle
        self.speed=0.01*(self.dx**2+self.dy**2)**0.5
        self.speed=limit(self.speed,-1,1)
        self.speed-=0.025*self.err_angle
        self.speed=limit(self.speed,-1,1)
        rotation_angle_rad = math.radians(self.rotation_angle)
        if self.speed>0:
            self.origin[0]+=self.speed*math.cos(rotation_angle_rad)
            self.origin[1]+=self.speed*math.sin(rotation_angle_rad)
        if self.speed<0:
            self.origin[0]+=self.speed*math.cos(rotation_angle_rad)
            self.origin[1]+=self.speed*math.sin(rotation_angle_rad)
        self.rotation_angle+=0.05*self.err_angle
        # print(self.err_angle)

    def Avoiding_obstacles(self,cars_all):
        for i in range(len(cars_all)):
            car=cars_all[i]
            L=distans_point2(car.origin,self.origin)
            # print(L)
            if L>1:
                self.dx-=(car.origin[0]-self.origin[0])*(10/L)
                self.dy-=(car.origin[1]-self.origin[1])*(10/L)

    def show(self,img):
        self.dx=0
        self.dy=0
        result1 = coordinate_transform(self.point1, self.origin, self.rotation_angle)
        result2 = coordinate_transform(self.point2, self.origin, self.rotation_angle)
        result3 = coordinate_transform(self.point3, self.origin, self.rotation_angle)
        pts = np.array([result1,result2,result3], np.int32)
        pts = pts.reshape((-1,1,2))
        cv.polylines(img,[pts],True,(0,255,255))
        return img



def mouse_callback(event, x, y, flags, param):
    global obj_point
    if event == cv2.EVENT_MOUSEMOVE:
        obj_point[0]=x
        obj_point[1]=y
        print("鼠标位置：", x, y)

# 创建一个窗口
cv2.namedWindow("image")
# 绑定鼠标事件和回调函数
cv2.setMouseCallback("image", mouse_callback)
obj_point=[100,200]
cars_all=[]
cars_all.append(cars([100,100],90))
cars_all.append(cars([150,110],90))
for i in range(50):
    cars_all.append(cars([random.randint(0,1000),random.randint(0,1000)],random.randint(-180,180)))
while 1:
    img=np.zeros((1000,1000,3),np.uint8)
    for i in range(len(cars_all)):
        car=cars_all[i]
        img=car.show(img)
        car.Avoiding_obstacles(cars_all)
        car.run(obj_point)
        
    cv.circle(img,obj_point, 3, (0,0,255), -1)
    cv2.imshow("image", img)
    if cv.waitKey(1) ==27:
        break
cv.waitKey(1) 
cv.destroyAllWindows()

鼠标位置： 9 228
鼠标位置： 18 230
鼠标位置： 29 234
鼠标位置： 40 236
鼠标位置： 51 238
鼠标位置： 60 240
鼠标位置： 70 242
鼠标位置： 76 243
鼠标位置： 83 245
鼠标位置： 96 247
鼠标位置： 102 248
鼠标位置： 106 249
鼠标位置： 109 249
鼠标位置： 117 251
鼠标位置： 130 253
鼠标位置： 147 255
鼠标位置： 170 258
鼠标位置： 190 261
鼠标位置： 200 263
鼠标位置： 202 263
鼠标位置： 205 263
鼠标位置： 205 263
鼠标位置： 206 263
鼠标位置： 206 263
鼠标位置： 207 263
鼠标位置： 207 264
鼠标位置： 207 264
鼠标位置： 208 264
鼠标位置： 210 265
鼠标位置： 210 265
鼠标位置： 211 266
鼠标位置： 212 267
鼠标位置： 213 268
鼠标位置： 213 269
鼠标位置： 213 270
鼠标位置： 214 271
鼠标位置： 214 272
鼠标位置： 214 274
鼠标位置： 216 275
鼠标位置： 217 276
鼠标位置： 217 278
鼠标位置： 218 278
鼠标位置： 219 280
鼠标位置： 220 282
鼠标位置： 224 284
鼠标位置： 229 288
鼠标位置： 234 291
鼠标位置： 237 291
鼠标位置： 238 291
鼠标位置： 239 291
鼠标位置： 240 289
鼠标位置： 242 288
鼠标位置： 245 285
鼠标位置： 248 283
鼠标位置： 252 280
鼠标位置： 260 273
鼠标位置： 260 273
鼠标位置： 266 267
鼠标位置： 269 263
鼠标位置： 272 260
鼠标位置： 274 257
鼠标位置： 276 253
鼠标位置： 278 250
鼠标位置： 279 246
鼠标位置： 280 244
鼠标位置： 280 244
鼠标位置： 258 232
鼠标位置： 243 227
鼠标位置： 234 225
鼠标位置： 230 224
鼠标位置： 224 223
鼠标位置： 222 222
鼠标位

In [ ]:
# if __name__ == "__main__":
#     # 调用摄像头 0为默认摄像头
#     fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # 定义视频编码器
#     out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (1000, 1000))  # 保存的文件名为output.avi，帧率为20，分辨率为(640, 480)
#     cap = cv2.VideoCapture(0)
#     obj_point=[100,150]
#     carmuch=[]
#     for i in range(50):
#         carmuch.append(cars([random.randint(0,1000),random.randint(0,1000)],random.randint(-180,180)))

#     # 通过循环将每一帧的图片读出来
#     while True:
#         # read方法返回两个参数
#         # success 判断摄像头是否打开成功，img 为读取的每一帧的图像对象
#         success, img = cap.read()
    
#         if not success:
#             print('摄像头打开失败')
#             break
    
#         # 0为开启上下镜像 1为开启左右镜像 -1为左右并上下镜像
#         img = cv2.flip(img, 1)
#         Gesture_results=Gesture_recognition(img)
#         if Gesture_results:
#             for i in Gesture_results[1]:
#                 cv.circle(img,(int(i.x*img.shape[1]),int(i.y*img.shape[0])), 3, (0,0,255), -1)
#                 obj_point=(int(i.x*img.shape[1]),int(i.y*img.shape[0]))
#         # imshow方法展示窗口，第一个参数为窗口的名字，第二个参数为帧数
#         imag=np.zeros((1000,1000,3),np.uint8)
#         for i in carmuch:
#             imag=i.show(imag)
#             i.run(obj_point)
  
#         cv.circle(imag,obj_point, 3, (0,0,255), -1)
#         out.write(imag)
#         cv2.imshow("sa", imag)
#         # cv2.imshow("frame", img)
#         # 延迟一毫秒

#         if cv.waitKey(1) == 27:
#             break
        
#     # 完成所有操作后，释放捕获器
#     cap.release()
#     out.release()
#     cv.destroyAllWindows()